In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/BDM25/csv_data/

appearances.csv   delta_data	    persistent_data	    transfers.csv
club_games.csv	  game_events.csv   persistent_data_Jintao
clubs.csv	  game_lineups.csv  players.csv
competitions.csv  games.csv	    player_valuations.csv


**Trusted Zone**

For CSV

In [ ]:
from pyspark.sql import SparkSession
import os

# Start a Spark session
spark = SparkSession.builder \
    .appName("TrustedZoneFootballExtended") \
    .getOrCreate()

# Define the base folder paths
data_path = "/content/drive/MyDrive/Colab Notebooks/BDM25/csv_data/"
trusted_dir = "/content/drive/MyDrive/Colab Notebooks/BDM25/trusted/"


In [ ]:
# Define the mapping of dataset names to file names
csv_files = {
    "players": "players.csv",
    "games": "games.csv",
    "appearances": "appearances.csv",
    "club_games": "club_games.csv",
    "clubs": "clubs.csv",
    "competitions": "competitions.csv",
    "game_events": "game_events.csv",
    "game_lineups": "game_lineups.csv",
    "player_valuations": "player_valuations.csv",
    "transfers": "transfers.csv"
}

# Load each CSV into a Spark DataFrame
dfs = {}
for name, file in csv_files.items():
    path = os.path.join(data_path, file)
    dfs[name] = spark.read.csv(path, header=True, inferSchema=True)


In [ ]:
dfs["players"] = dfs["players"].dropna(subset=["player_id"]).dropDuplicates(["player_id"])
dfs["games"] = dfs["games"].dropna(subset=["game_id"]).dropDuplicates(["game_id"])
dfs["appearances"] = dfs["appearances"].dropna(subset=["player_id", "game_id"])
dfs["club_games"] = dfs["club_games"].dropna(subset=["game_id", "club_id"])
dfs["clubs"] = dfs["clubs"].dropna(subset=["club_id"]).dropDuplicates(["club_id"])
dfs["competitions"] = dfs["competitions"].dropna(subset=["competition_id"]).dropDuplicates(["competition_id"])
dfs["game_events"] = dfs["game_events"].dropna(subset=["game_id"])
dfs["game_lineups"] = dfs["game_lineups"].dropna(subset=["game_id", "player_id"])
dfs["player_valuations"] = dfs["player_valuations"].dropna(subset=["player_id", "date"])
dfs["transfers"] = dfs["transfers"].dropna(subset=["player_id", "transfer_date"])


In [ ]:
# Remove appearances with invalid player_id
invalid_appearances = dfs["appearances"].join(
    dfs["players"].select("player_id"), on="player_id", how="left_anti"
)

# Save invalid rows for debugging
invalid_appearances.write.mode("overwrite").parquet(f"{trusted_dir}invalid_appearances.parquet")

# Keep only valid appearances
dfs["appearances"] = dfs["appearances"].join(
    dfs["players"].select("player_id"), on="player_id", how="inner"
)


In [ ]:
for name, df in dfs.items():
    df.write.mode("overwrite").parquet(f"{trusted_dir}{name}.parquet")


In [ ]:
con.execute("DROP TABLE IF EXISTS players;")
con.execute("DROP TABLE IF EXISTS games;")
con.execute("DROP TABLE IF EXISTS appearances;")

In [ ]:
import duckdb

# Connect to DuckDB file
con = duckdb.connect(f"{trusted_dir}trusted_zone.duckdb")

# Ensure the parquet extension is available
con.execute("INSTALL parquet; LOAD parquet;")

# Load each parquet dataset into a DuckDB table
for name in dfs:
    con.execute(f"DROP TABLE IF EXISTS {name};")
    con.execute(f"""
        CREATE TABLE {name} AS
        SELECT * FROM parquet_scan('{trusted_dir}{name}.parquet/*.parquet');
    """)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.execute("SELECT COUNT(*) FROM players").fetchall()

[(32601,)]

For Video

In [ ]:
!pip install opencv-python

In [ ]:
from pyspark.sql import SparkSession
import cv2
import os


spark = SparkSession.builder.appName("VideoMetadata").getOrCreate()
sc = spark.sparkContext


video_path = "/content/drive/MyDrive/Colab Notebooks/BDM25/video_data/Bundesliga1.mp4"

def extract_video_metadata(path):
    if not os.path.exists(path):
        return (path, "Not Found", None, None)

    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        return (path, "Cannot open", None, None)


    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_count / fps if fps else 0


    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    return (path, "OK", round(duration, 2), f"{width}x{height}")


video_files_rdd = sc.parallelize([video_path])

metadata_rdd = video_files_rdd.map(extract_video_metadata)

#
metadata_df = metadata_rdd.toDF(["path", "status", "duration_sec", "resolution"])

#
metadata_df.show(truncate=False)


+-----------------------------------------------------------------------+------+------------+----------+
|path                                                                   |status|duration_sec|resolution|
+-----------------------------------------------------------------------+------+------------+----------+
|/content/drive/MyDrive/Colab Notebooks/BDM25/video_data/Bundesliga1.mp4|OK    |123.04      |1920x1080 |
+-----------------------------------------------------------------------+------+------------+----------+



For JSON

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col

spark = SparkSession.builder.appName("FlattenJSON").getOrCreate()


In [ ]:
json_path = "/content/drive/MyDrive/Colab Notebooks/BDM25/video_data/Bundesliga1.json"

df = spark.read.option("multiline", "true").json(json_path)
df.printSchema()


root
 |-- author: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- published_at: string (nullable = true)
 |-- text: string (nullable = true)

